In [ ]:
## This file implements neural networks with/without dropout and regularizer for p0017SpresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0017SpresabsSTCC_qual.csv')
df.shape

(253, 153)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
      ..
248    1
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTAATACATAT,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,TTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTTGCACA,...,group_1687,group_3441,group_4225,group_4420,group_7795,group_8042,group_8892,ST,CC,pheno
0,107,1,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,5,5,0
1,109,1,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,1,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,1,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,5,5,0
4,120337,1,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,5,5,0


In [6]:
df['pheno'].value_counts()

0    216
1     35
2      2
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 152)

In [9]:
df_clean.head()

,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTAATACATAT,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,TTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTTGCACA,TTCAAGAAGGAGA,...,group_1687,group_3441,group_4225,group_4420,group_7795,group_8042,group_8892,ST,CC,pheno
0,1,0,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,5,5,0
1,1,0,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,8,8,0
2,1,1,1,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,5,5,1
3,1,0,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,5,5,0
4,1,0,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,5,5,0


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 152) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 216), (1, 216), (2, 216)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,312,1
1,CFBRSa27,0
2,BCH-SA-01,1
3,GA27,1
4,NRS209,2
...,...,...
190,NRS209,2
191,NRS235,0
192,NRS240,1
193,NRS110,2


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 302us/step - loss: 12.5503 - accuracy: 0.4525 - val_loss: 8.7655 - val_accuracy: 0.3487
Epoch 2/100
453/453 [==============================] - 0s 75us/step - loss: 6.5873 - accuracy: 0.4636 - val_loss: 3.9965 - val_accuracy: 0.3897
Epoch 3/100
453/453 [==============================] - 0s 84us/step - loss: 3.3724 - accuracy: 0.5519 - val_loss: 1.9124 - val_accuracy: 0.5128
Epoch 4/100
453/453 [==============================] - 0s 93us/step - loss: 1.3998 - accuracy: 0.5453 - val_loss: 0.9290 - val_accuracy: 0.5949
Epoch 5/100
453/453 [==============================] - 0s 86us/step - loss: 0.7768 - accuracy: 0.7108 - val_loss: 0.7546 - val_accuracy: 0.7538
Epoch 6/100
453/453 [==============================] - 0s 79us/step - loss: 0.6967 - accuracy: 0.6777 - val_loss: 0.5473 - val_accuracy: 0.8154
Epoch 7/100
453/453 [==============================] - 0s 90us/step - loss: 0.5235 - acc

453/453 [==============================] - 0s 93us/step - loss: 0.1298 - accuracy: 0.9603 - val_loss: 0.2599 - val_accuracy: 0.9026
Epoch 58/100
453/453 [==============================] - 0s 94us/step - loss: 0.1367 - accuracy: 0.9536 - val_loss: 0.2807 - val_accuracy: 0.9026
Epoch 59/100
453/453 [==============================] - 0s 89us/step - loss: 0.2756 - accuracy: 0.9536 - val_loss: 0.5114 - val_accuracy: 0.8872
Epoch 60/100
453/453 [==============================] - 0s 85us/step - loss: 0.1777 - accuracy: 0.9382 - val_loss: 0.2978 - val_accuracy: 0.9077
Epoch 61/100
453/453 [==============================] - 0s 88us/step - loss: 0.1368 - accuracy: 0.9514 - val_loss: 0.2498 - val_accuracy: 0.8974
Epoch 62/100
453/453 [==============================] - 0s 94us/step - loss: 0.1440 - accuracy: 0.9426 - val_loss: 0.2605 - val_accuracy: 0.9179
Epoch 63/100
453/453 [==============================] - 0s 208us/step - loss: 0.2898 - accuracy: 0.9205 - val_loss: 0.3388 - val_accuracy: 0.88

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.104641). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


Epoch 70/100
453/453 [==============================] - 0s 171us/step - loss: 0.1174 - accuracy: 0.9536 - val_loss: 0.2653 - val_accuracy: 0.9231
Epoch 71/100
453/453 [==============================] - 0s 96us/step - loss: 0.1130 - accuracy: 0.9581 - val_loss: 0.2576 - val_accuracy: 0.8923
Epoch 72/100
453/453 [==============================] - 0s 171us/step - loss: 0.1077 - accuracy: 0.9625 - val_loss: 0.2605 - val_accuracy: 0.9385
Epoch 73/100
453/453 [==============================] - 0s 146us/step - loss: 0.1063 - accuracy: 0.9669 - val_loss: 0.2508 - val_accuracy: 0.9128
Epoch 74/100
453/453 [==============================] - 0s 94us/step - loss: 0.1051 - accuracy: 0.9603 - val_loss: 0.2727 - val_accuracy: 0.9282
Epoch 75/100
453/453 [==============================] - 0s 145us/step - loss: 0.1095 - accuracy: 0.9581 - val_loss: 0.2491 - val_accuracy: 0.9282
Epoch 76/100
453/453 [==============================] - 0s 102us/step - loss: 0.1036 - accuracy: 0.9669 - val_loss: 0.2638 - v

In [29]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

195/195 [==============================] - 0s 220us/step
over-sampling test accuracy: 92.82%


In [47]:
pred = model1_over.predict_classes(X_test_over)
pred

array([1, 0, 1, 1, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 1, 2, 1, 1,
       2, 1, 1, 1, 0, 0, 0, 0, 2, 1, 2, 2, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 2, 0, 0, 1, 1, 0, 1, 1, 0, 2, 0, 1, 2, 2, 2, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 2, 2, 1, 2, 2, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 0, 2,
       1, 2, 1, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 1, 0, 1, 0, 2, 1, 2,
       2, 1, 1, 1, 1, 2, 2, 2, 1, 0, 2, 1, 1, 2, 0, 0, 1, 0, 0, 2, 0, 2,
       0, 0, 0, 2, 2, 0, 2, 1, 0, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 0, 2, 1, 2, 2, 2, 1, 1, 2, 1, 0, 2,
       1, 2, 2, 1, 2, 1, 0, 2, 2, 2, 2, 2, 1, 1, 2, 0, 1, 2, 1])

In [48]:
dat['pred'] = pred
dat

,0,test,pred
0,312,1,1
1,CFBRSa27,0,0
2,BCH-SA-01,1,1
3,GA27,1,1
4,NRS209,2,2
...,...,...,...
190,NRS209,2,2
191,NRS235,0,0
192,NRS240,1,1
193,NRS110,2,2


In [49]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [50]:
dat_proba1

,0,1,2
0,0.085174,0.914432,3.944692e-04
1,0.999989,0.000011,4.954430e-09
2,0.087625,0.912372,2.741069e-06
3,0.327130,0.672864,5.658259e-06
4,0.000713,0.005007,9.942797e-01
...,...,...,...
190,0.000713,0.005007,9.942797e-01
191,0.982151,0.017841,8.293405e-06
192,0.034630,0.965175,1.948009e-04
193,0.000889,0.000444,9.986665e-01


In [51]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [52]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p17spST.csv", index = False,
         header=None)

In [33]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 201us/step - loss: 0.0959 - accuracy: 0.9647 - val_loss: 0.2845 - val_accuracy: 0.9282
Epoch 2/100
453/453 [==============================] - 0s 163us/step - loss: 0.0961 - accuracy: 0.9558 - val_loss: 0.2522 - val_accuracy: 0.9436
Epoch 3/100
453/453 [==============================] - 0s 132us/step - loss: 0.0932 - accuracy: 0.9536 - val_loss: 0.2661 - val_accuracy: 0.9231
Epoch 4/100
453/453 [==============================] - 0s 141us/step - loss: 0.0891 - accuracy: 0.9536 - val_loss: 0.2551 - val_accuracy: 0.9385
Epoch 5/100
453/453 [==============================] - 0s 151us/step - loss: 0.0872 - accuracy: 0.9647 - val_loss: 0.2750 - val_accuracy: 0.9385
Epoch 6/100
453/453 [==============================] - 0s 145us/step - loss: 0.0872 - accuracy: 0.9581 - val_loss: 0.2572 - val_accuracy: 0.9385
Epoch 7/100
453/453 [==============================] - 0s 124us/step - loss: 0.0888 

In [34]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 97.05%


In [18]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [19]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [20]:
y_prob = df_proba[df_proba['phage']=='p0017SpresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[8.51738400e-02, 9.14431630e-01, 3.94469160e-04],
       [9.99988800e-01, 1.12411320e-05, 4.95443000e-09],
       [8.76250500e-02, 9.12372200e-01, 2.74106950e-06],
       [3.27129750e-01, 6.72864500e-01, 5.65825940e-06],
       [7.13185600e-04, 5.00711500e-03, 9.94279740e-01],
       [8.51738400e-02, 9.14431630e-01, 3.94469160e-04],
       [1.30223260e-06, 9.99764740e-01, 2.33929780e-04],
       [5.22255660e-01, 4.36461660e-01, 4.12826900e-02],
       [9.99995470e-01, 4.52827770e-06, 1.33635090e-08],
       [9.99942540e-01, 5.74940820e-05, 4.88934300e-09],
       [6.45482540e-02, 9.35451600e-01, 1.31027520e-07],
       [4.06707260e-04, 9.99201600e-01, 3.91805630e-04],
       [9.99709550e-01, 2.90217780e-04, 2.59826800e-07],
       [9.99980570e-01, 1.94701020e-05, 4.08222340e-09],
       [9.99605600e-01, 3.94359400e-04, 1.53632670e-07],
       [9.99742700e-01, 2.57298930e-04, 5.37725670e-08],
       [7.13185600e-04, 5.00711500e-03, 9.94279740e-01],
       [9.99986300e-01, 1.37253

In [21]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [22]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.973491124260355

In [23]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [24]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.973491124260355

In [25]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [26]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [27]:
dat2

,0,test
0,NRS110,2
1,NRS254,1
2,BCH-SA-09,0
3,NRS177,0
4,GA27,1
...,...,...
190,NRS001,1
191,NRS209,2
192,NRS272,0
193,NRS110,2


In [28]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [29]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 412us/step - loss: 3.7170 - accuracy: 0.3664 - val_loss: 1.2922 - val_accuracy: 0.3590
Epoch 2/100
453/453 [==============================] - 0s 132us/step - loss: 1.2709 - accuracy: 0.5430 - val_loss: 1.0242 - val_accuracy: 0.5538
Epoch 3/100
453/453 [==============================] - 0s 127us/step - loss: 0.8559 - accuracy: 0.6380 - val_loss: 0.7750 - val_accuracy: 0.7231
Epoch 4/100
453/453 [==============================] - 0s 121us/step - loss: 0.7099 - accuracy: 0.7748 - val_loss: 0.5365 - val_accuracy: 0.8103
Epoch 5/100
453/453 [==============================] - 0s 125us/step - loss: 0.5296 - accuracy: 0.8035 - val_loss: 0.4708 - val_accuracy: 0.8205
Epoch 6/100
453/453 [==============================] - 0s 105us/step - loss: 0.4740 - accuracy: 0.8079 - val_loss: 0.4458 - val_accuracy: 0.8103
Epoch 7/100
453/453 [==============================] - 0s 108us/step - loss: 0.4974 

Epoch 57/100
453/453 [==============================] - 0s 121us/step - loss: 0.1389 - accuracy: 0.9581 - val_loss: 0.2253 - val_accuracy: 0.9282
Epoch 58/100
453/453 [==============================] - 0s 181us/step - loss: 0.1597 - accuracy: 0.9404 - val_loss: 0.2379 - val_accuracy: 0.9077
Epoch 59/100
453/453 [==============================] - 0s 184us/step - loss: 0.1353 - accuracy: 0.9536 - val_loss: 0.2501 - val_accuracy: 0.8872
Epoch 60/100
453/453 [==============================] - 0s 191us/step - loss: 0.1375 - accuracy: 0.9492 - val_loss: 0.2255 - val_accuracy: 0.9282
Epoch 61/100
453/453 [==============================] - 0s 87us/step - loss: 0.1496 - accuracy: 0.9581 - val_loss: 0.2578 - val_accuracy: 0.8923
Epoch 62/100
453/453 [==============================] - 0s 90us/step - loss: 0.1344 - accuracy: 0.9625 - val_loss: 0.2369 - val_accuracy: 0.9128
Epoch 63/100
453/453 [==============================] - 0s 66us/step - loss: 0.1303 - accuracy: 0.9625 - val_loss: 0.2309 - va

In [62]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

195/195 [==============================] - 0s 69us/step
over-sampling test accuracy: 92.82%


In [60]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([2, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 2, 0, 1, 1, 0, 0, 0,
       2, 1, 0, 0, 1, 1, 0, 2, 0, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 0,
       2, 0, 0, 2, 2, 0, 1, 1, 1, 2, 1, 0, 0, 1, 2, 2, 2, 2, 1, 1, 1, 2,
       0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 2, 0, 1, 1, 1, 2, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 2, 0, 2, 1, 0, 0, 1, 0, 2, 2, 0, 2, 0, 1, 1, 2, 1, 1,
       2, 2, 1, 2, 2, 1, 0, 0, 2, 1, 1, 1, 2, 2, 0, 2, 0, 2, 2, 0, 1, 2,
       1, 0, 1, 2, 2, 0, 1, 0, 2, 1, 1, 0, 2, 1, 2, 1, 0, 2, 1, 2, 1, 1,
       1, 2, 1, 2, 0, 1, 0, 1, 2, 0, 2, 1, 2, 0, 0, 2, 1, 2, 2, 1, 1, 2,
       1, 2, 0, 2, 1, 0, 0, 0, 1, 2, 0, 2, 0, 2, 1, 2, 1, 2, 0])

In [61]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,NRS110,2,2
1,NRS254,1,1
2,BCH-SA-09,0,1
3,NRS177,0,0
4,GA27,1,1
...,...,...,...
190,NRS001,1,1
191,NRS209,2,2
192,NRS272,0,1
193,NRS110,2,2


In [62]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [63]:
dat_proba2

,0,1,2
0,0.000125,0.000399,9.994769e-01
1,0.000057,0.999626,3.175176e-04
2,0.293587,0.706319,9.398517e-05
3,0.998093,0.001578,3.293598e-04
4,0.130694,0.869306,3.158125e-07
...,...,...,...
190,0.060341,0.939657,1.892719e-06
191,0.000835,0.001227,9.979379e-01
192,0.314853,0.622859,6.228807e-02
193,0.000125,0.000399,9.994769e-01


In [64]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [65]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p17spST.csv", index = False,
         header=None)

In [66]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 131us/step - loss: 0.0927 - accuracy: 0.9713 - val_loss: 0.2148 - val_accuracy: 0.9385
Epoch 2/100
453/453 [==============================] - 0s 147us/step - loss: 0.0938 - accuracy: 0.9713 - val_loss: 0.2843 - val_accuracy: 0.9077
Epoch 3/100
453/453 [==============================] - 0s 146us/step - loss: 0.0943 - accuracy: 0.9691 - val_loss: 0.2120 - val_accuracy: 0.9385
Epoch 4/100
453/453 [==============================] - 0s 152us/step - loss: 0.0912 - accuracy: 0.9713 - val_loss: 0.2242 - val_accuracy: 0.9282
Epoch 5/100
453/453 [==============================] - 0s 131us/step - loss: 0.0896 - accuracy: 0.9757 - val_loss: 0.2262 - val_accuracy: 0.9282
Epoch 6/100
453/453 [==============================] - 0s 127us/step - loss: 0.0896 - accuracy: 0.9691 - val_loss: 0.2153 - val_accuracy: 0.9385
Epoch 7/100
453/453 [==============================] - 0s 144us/step - loss: 0.0963 

Epoch 57/100
453/453 [==============================] - 0s 117us/step - loss: 0.0618 - accuracy: 0.9868 - val_loss: 0.2595 - val_accuracy: 0.9179
Epoch 58/100
453/453 [==============================] - 0s 100us/step - loss: 0.0712 - accuracy: 0.9757 - val_loss: 0.2299 - val_accuracy: 0.9436
Epoch 59/100
453/453 [==============================] - 0s 105us/step - loss: 0.0654 - accuracy: 0.9823 - val_loss: 0.2849 - val_accuracy: 0.9077
Epoch 60/100
453/453 [==============================] - 0s 146us/step - loss: 0.0670 - accuracy: 0.9801 - val_loss: 0.2484 - val_accuracy: 0.9179
Epoch 61/100
453/453 [==============================] - 0s 155us/step - loss: 0.0645 - accuracy: 0.9823 - val_loss: 0.2425 - val_accuracy: 0.9282
Epoch 62/100
453/453 [==============================] - 0s 124us/step - loss: 0.0613 - accuracy: 0.9801 - val_loss: 0.2552 - val_accuracy: 0.9179
Epoch 63/100
453/453 [==============================] - 0s 117us/step - loss: 0.0632 - accuracy: 0.9801 - val_loss: 0.2691 -

In [67]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 97.33%


In [30]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [31]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [32]:
y_prob2 = df_proba2[df_proba2['phage']=='p0017SpresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[1.24710800e-04, 3.98505000e-04, 9.99476850e-01],
       [5.68281360e-05, 9.99625560e-01, 3.17517640e-04],
       [2.93586900e-01, 7.06319150e-01, 9.39851700e-05],
       [9.98092950e-01, 1.57770400e-03, 3.29359800e-04],
       [1.30693580e-01, 8.69306100e-01, 3.15812460e-07],
       [9.99999170e-01, 8.18356170e-07, 5.11358960e-08],
       [1.67183470e-01, 8.32792900e-01, 2.36414290e-05],
       [2.11697610e-02, 9.78829500e-01, 7.42307860e-07],
       [9.92704500e-01, 7.21716230e-03, 7.83993300e-05],
       [1.24710800e-04, 3.98505000e-04, 9.99476850e-01],
       [6.03414070e-02, 9.39656600e-01, 1.89271910e-06],
       [9.99987100e-01, 1.29095390e-05, 2.82659890e-08],
       [8.35038100e-04, 1.22713270e-03, 9.97937860e-01],
       [8.35038100e-04, 1.22713270e-03, 9.97937860e-01],
       [1.18528580e-05, 9.99873900e-01, 1.14248964e-04],
       [1.24710800e-04, 3.98505000e-04, 9.99476850e-01],
       [1.00000000e+00, 3.69256000e-08, 4.17103260e-11],
       [6.03414070e-02, 9.39656

In [33]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9839447731755424

In [34]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9839447731755424

In [35]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [36]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [37]:
dat3

,0,test
0,NRS249,1
1,NRS172,1
2,NRS209,2
3,NRS108,1
4,NRS209,2
...,...,...
190,NRS209,2
191,NRS110,2
192,NRS255,1
193,NRS175,1


In [38]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [39]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [71]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 328us/step - loss: 4.5625 - accuracy: 0.4790 - val_loss: 6.0899 - val_accuracy: 0.4923
Epoch 2/100
453/453 [==============================] - 0s 83us/step - loss: 1.5184 - accuracy: 0.6203 - val_loss: 1.2294 - val_accuracy: 0.6513
Epoch 3/100
453/453 [==============================] - 0s 86us/step - loss: 1.1417 - accuracy: 0.5673 - val_loss: 1.3343 - val_accuracy: 0.6821
Epoch 4/100
453/453 [==============================] - 0s 90us/step - loss: 0.9137 - accuracy: 0.7042 - val_loss: 0.5758 - val_accuracy: 0.7385
Epoch 5/100
453/453 [==============================] - 0s 86us/step - loss: 0.6168 - accuracy: 0.7285 - val_loss: 0.4928 - val_accuracy: 0.7282
Epoch 6/100
453/453 [==============================] - 0s 87us/step - loss: 0.5193 - accuracy: 0.7638 - val_loss: 0.6887 - val_accuracy: 0.7692
Epoch 7/100
453/453 [==============================] - 0s 105us/step - loss: 0.4888 - acc

453/453 [==============================] - 0s 84us/step - loss: 0.1633 - accuracy: 0.9294 - val_loss: 0.1534 - val_accuracy: 0.9590
Epoch 58/100
453/453 [==============================] - 0s 93us/step - loss: 0.1634 - accuracy: 0.9426 - val_loss: 0.1518 - val_accuracy: 0.9641
Epoch 59/100
453/453 [==============================] - 0s 84us/step - loss: 0.1709 - accuracy: 0.9448 - val_loss: 0.1777 - val_accuracy: 0.9385
Epoch 60/100
453/453 [==============================] - 0s 141us/step - loss: 0.1953 - accuracy: 0.9360 - val_loss: 0.1571 - val_accuracy: 0.9436
Epoch 61/100
453/453 [==============================] - 0s 148us/step - loss: 0.1618 - accuracy: 0.9360 - val_loss: 0.1519 - val_accuracy: 0.9538
Epoch 62/100
453/453 [==============================] - 0s 118us/step - loss: 0.1540 - accuracy: 0.9360 - val_loss: 0.1566 - val_accuracy: 0.9385
Epoch 63/100
453/453 [==============================] - 0s 87us/step - loss: 0.1544 - accuracy: 0.9404 - val_loss: 0.1472 - val_accuracy: 0.

In [75]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

195/195 [==============================] - 0s 77us/step
over-sampling test accuracy: 95.90%


In [73]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([1, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 2, 0, 2, 0, 0, 1, 2,
       1, 0, 2, 0, 1, 0, 0, 0, 0, 0, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 1, 0,
       2, 2, 2, 2, 2, 1, 2, 2, 1, 0, 2, 2, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 2, 2, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 2, 1, 0, 2, 0, 2, 1, 1, 2,
       1, 2, 0, 0, 2, 0, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 0, 2, 1, 2, 0, 1,
       2, 0, 1, 0, 1, 1, 2, 2, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 2, 2, 0, 1,
       2, 2, 1, 2, 2, 2, 0, 1, 1, 0, 2, 1, 1, 1, 2, 0, 2, 2, 0, 2, 1, 0,
       1, 2, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2,
       0, 2, 1, 1, 2, 0, 2, 1, 1, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1])

In [74]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS249,1,1
1,NRS172,1,1
2,NRS209,2,2
3,NRS108,1,1
4,NRS209,2,2
...,...,...,...
190,NRS209,2,2
191,NRS110,2,2
192,NRS255,1,1
193,NRS175,1,1


In [75]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [76]:
dat_proba3

,0,1,2
0,0.026318,0.973601,0.000081
1,0.000043,0.999952,0.000005
2,0.000974,0.002337,0.996689
3,0.000006,0.999980,0.000014
4,0.000974,0.002337,0.996689
...,...,...,...
190,0.000974,0.002337,0.996689
191,0.001073,0.000155,0.998772
192,0.000557,0.994291,0.005152
193,0.268605,0.731393,0.000002


In [77]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [78]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p17spST.csv", index = False,
         header=None)

In [79]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 128us/step - loss: 0.1009 - accuracy: 0.9647 - val_loss: 0.1287 - val_accuracy: 0.9487
Epoch 2/100
453/453 [==============================] - 0s 134us/step - loss: 0.1051 - accuracy: 0.9581 - val_loss: 0.1121 - val_accuracy: 0.9487
Epoch 3/100
453/453 [==============================] - 0s 119us/step - loss: 0.1027 - accuracy: 0.9603 - val_loss: 0.1124 - val_accuracy: 0.9538
Epoch 4/100
453/453 [==============================] - 0s 94us/step - loss: 0.1064 - accuracy: 0.9625 - val_loss: 0.1268 - val_accuracy: 0.9487
Epoch 5/100
453/453 [==============================] - 0s 104us/step - loss: 0.0999 - accuracy: 0.9691 - val_loss: 0.1226 - val_accuracy: 0.9436
Epoch 6/100
453/453 [==============================] - 0s 101us/step - loss: 0.0962 - accuracy: 0.9581 - val_loss: 0.1155 - val_accuracy: 0.9590
Epoch 7/100
453/453 [==============================] - 0s 100us/step - loss: 0.0977 -

Epoch 57/100
453/453 [==============================] - 0s 178us/step - loss: 0.0657 - accuracy: 0.9647 - val_loss: 0.1025 - val_accuracy: 0.9641
Epoch 58/100
453/453 [==============================] - 0s 117us/step - loss: 0.0632 - accuracy: 0.9757 - val_loss: 0.1026 - val_accuracy: 0.9641
Epoch 59/100
453/453 [==============================] - 0s 401us/step - loss: 0.0662 - accuracy: 0.9603 - val_loss: 0.1048 - val_accuracy: 0.9641
Epoch 60/100
453/453 [==============================] - 0s 176us/step - loss: 0.0635 - accuracy: 0.9801 - val_loss: 0.0985 - val_accuracy: 0.9538
Epoch 61/100
 32/453 [=>............................] - ETA: 0s - loss: 0.0707 - accuracy: 0.9375

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.100914). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


453/453 [==============================] - 0s 128us/step - loss: 0.0636 - accuracy: 0.9823 - val_loss: 0.1213 - val_accuracy: 0.9590
Epoch 62/100
453/453 [==============================] - 0s 92us/step - loss: 0.0786 - accuracy: 0.9647 - val_loss: 0.0952 - val_accuracy: 0.9590
Epoch 63/100
453/453 [==============================] - 0s 127us/step - loss: 0.0655 - accuracy: 0.9735 - val_loss: 0.1003 - val_accuracy: 0.9487
Epoch 64/100
453/453 [==============================] - 0s 133us/step - loss: 0.0636 - accuracy: 0.9757 - val_loss: 0.1051 - val_accuracy: 0.9641
Epoch 65/100
453/453 [==============================] - 0s 138us/step - loss: 0.0670 - accuracy: 0.9713 - val_loss: 0.1601 - val_accuracy: 0.9385
Epoch 66/100
453/453 [==============================] - 0s 181us/step - loss: 0.0621 - accuracy: 0.9735 - val_loss: 0.0968 - val_accuracy: 0.9487
Epoch 67/100
453/453 [==============================] - 0s 192us/step - loss: 0.0637 - accuracy: 0.9801 - val_loss: 0.1054 - val_accuracy:

In [80]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 97.08%


In [40]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [41]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [42]:
y_prob3 = df_proba3[df_proba3['phage']=='p0017SpresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[2.63180710e-02, 9.73601160e-01, 8.07495450e-05],
       [4.29265560e-05, 9.99952200e-01, 4.87588930e-06],
       [9.73985530e-04, 2.33714930e-03, 9.96688900e-01],
       [5.92387050e-06, 9.99980450e-01, 1.36290280e-05],
       [9.73985530e-04, 2.33714930e-03, 9.96688900e-01],
       [4.94338750e-01, 5.05638960e-01, 2.23278100e-05],
       [9.90479050e-01, 9.51550200e-03, 5.48244400e-06],
       [2.63180710e-02, 9.73601160e-01, 8.07495450e-05],
       [9.29314430e-01, 7.06854700e-02, 1.28846000e-07],
       [4.29265560e-05, 9.99952200e-01, 4.87588930e-06],
       [5.02546200e-01, 4.97434650e-01, 1.90863280e-05],
       [9.99141700e-01, 8.58285350e-04, 2.29634800e-10],
       [4.29265560e-05, 9.99952200e-01, 4.87588930e-06],
       [5.57202800e-04, 9.94291070e-01, 5.15168700e-03],
       [4.59480020e-01, 5.40519100e-01, 8.91815600e-07],
       [1.07313940e-03, 1.55146950e-04, 9.98771700e-01],
       [5.59421500e-01, 4.39948800e-01, 6.29719000e-04],
       [1.07313940e-03, 1.55146

In [43]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9927021696252466

In [44]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9927021696252466

In [45]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [46]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [47]:
dat4

,0,test
0,NRS241,1
1,BCH-SA-01,1
2,NRS219,1
3,NRS209,2
4,NRS001,1
...,...,...
190,GA15,1
191,NRS246,1
192,115,1
193,312,1


In [48]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [49]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [84]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 290us/step - loss: 16.2410 - accuracy: 0.2980 - val_loss: 9.9294 - val_accuracy: 0.3385
Epoch 2/100
453/453 [==============================] - 0s 54us/step - loss: 11.1890 - accuracy: 0.3974 - val_loss: 6.6034 - val_accuracy: 0.4513
Epoch 3/100
453/453 [==============================] - 0s 82us/step - loss: 7.0869 - accuracy: 0.4547 - val_loss: 3.6466 - val_accuracy: 0.5077
Epoch 4/100
453/453 [==============================] - 0s 79us/step - loss: 2.7689 - accuracy: 0.4636 - val_loss: 2.5197 - val_accuracy: 0.3897
Epoch 5/100
453/453 [==============================] - 0s 82us/step - loss: 1.8465 - accuracy: 0.3907 - val_loss: 1.0527 - val_accuracy: 0.4205
Epoch 6/100
453/453 [==============================] - 0s 95us/step - loss: 1.0844 - accuracy: 0.4636 - val_loss: 0.7405 - val_accuracy: 0.7026
Epoch 7/100
453/453 [==============================] - 0s 141us/step - loss: 0.8849 - a

In [97]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

195/195 [==============================] - 0s 118us/step
over-sampling test accuracy: 93.85%


In [86]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 1, 0, 2, 1, 2, 1, 1, 2, 2, 2, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 1,
       1, 2, 1, 0, 0, 0, 2, 1, 2, 1, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 1, 0,
       0, 0, 2, 2, 1, 1, 2, 0, 1, 2, 1, 2, 1, 0, 0, 1, 1, 1, 0, 1, 2, 0,
       1, 0, 0, 2, 1, 1, 2, 1, 1, 0, 2, 1, 0, 2, 1, 0, 2, 2, 2, 1, 2, 2,
       2, 2, 1, 0, 1, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 2, 0, 2, 1, 0,
       2, 2, 0, 2, 2, 1, 2, 2, 2, 1, 0, 0, 2, 2, 2, 1, 0, 1, 2, 0, 1, 1,
       1, 0, 1, 1, 2, 1, 1, 2, 2, 2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 2, 1,
       0, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2, 0, 0, 2, 0, 2, 1, 0, 0, 2, 0, 0,
       1, 2, 1, 0, 1, 2, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [87]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,NRS241,1,1
1,BCH-SA-01,1,1
2,NRS219,1,0
3,NRS209,2,2
4,NRS001,1,1
...,...,...,...
190,GA15,1,1
191,NRS246,1,1
192,115,1,1
193,312,1,1


In [88]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [89]:
dat_proba4

,0,1,2
0,0.072540,0.924324,3.135318e-03
1,0.145944,0.853712,3.438867e-04
2,0.772667,0.222506,4.826286e-03
3,0.008321,0.012965,9.787133e-01
4,0.373947,0.624066,1.986789e-03
...,...,...,...
190,0.387804,0.610818,1.377677e-03
191,0.441195,0.546824,1.198106e-02
192,0.031148,0.968094,7.574646e-04
193,0.224255,0.775738,6.864440e-06


In [90]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [91]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p17spST.csv", index = False,
         header=None)

In [101]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 86us/step - loss: 0.1752 - accuracy: 0.9404 - val_loss: 0.1970 - val_accuracy: 0.9333
Epoch 2/100
453/453 [==============================] - 0s 126us/step - loss: 0.1796 - accuracy: 0.9316 - val_loss: 0.2019 - val_accuracy: 0.9333
Epoch 3/100
453/453 [==============================] - 0s 82us/step - loss: 0.1797 - accuracy: 0.9338 - val_loss: 0.1931 - val_accuracy: 0.9282
Epoch 4/100
453/453 [==============================] - 0s 91us/step - loss: 0.1779 - accuracy: 0.9272 - val_loss: 0.1912 - val_accuracy: 0.9282
Epoch 5/100
453/453 [==============================] - 0s 73us/step - loss: 0.1767 - accuracy: 0.9294 - val_loss: 0.1865 - val_accuracy: 0.9436
Epoch 6/100
453/453 [==============================] - 0s 90us/step - loss: 0.1737 - accuracy: 0.9470 - val_loss: 0.1901 - val_accuracy: 0.9436
Epoch 7/100
453/453 [==============================] - 0s 82us/step - loss: 0.1694 - accu

Epoch 57/100
453/453 [==============================] - 0s 68us/step - loss: 0.2638 - accuracy: 0.9227 - val_loss: 0.1778 - val_accuracy: 0.9385
Epoch 58/100
453/453 [==============================] - 0s 81us/step - loss: 0.1378 - accuracy: 0.9536 - val_loss: 0.1843 - val_accuracy: 0.9385
Epoch 59/100
453/453 [==============================] - 0s 77us/step - loss: 0.1279 - accuracy: 0.9536 - val_loss: 0.1624 - val_accuracy: 0.9436
Epoch 60/100
453/453 [==============================] - 0s 104us/step - loss: 0.1223 - accuracy: 0.9603 - val_loss: 0.1620 - val_accuracy: 0.9333
Epoch 61/100
453/453 [==============================] - 0s 82us/step - loss: 0.1213 - accuracy: 0.9647 - val_loss: 0.1530 - val_accuracy: 0.9487
Epoch 62/100
453/453 [==============================] - 0s 95us/step - loss: 0.1166 - accuracy: 0.9669 - val_loss: 0.1529 - val_accuracy: 0.9487
Epoch 63/100
453/453 [==============================] - 0s 95us/step - loss: 0.1177 - accuracy: 0.9647 - val_loss: 0.1521 - val_a

In [102]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 95.34%


In [50]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [51]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [52]:
y_prob4 = df_proba4[df_proba4['phage']=='p0017SpresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[7.25404840e-02, 9.24324150e-01, 3.13531820e-03],
       [1.45943580e-01, 8.53712500e-01, 3.43886700e-04],
       [7.72667400e-01, 2.22506400e-01, 4.82628630e-03],
       [8.32148300e-03, 1.29651830e-02, 9.78713330e-01],
       [3.73947470e-01, 6.24065760e-01, 1.98678930e-03],
       [8.32148300e-03, 1.29651830e-02, 9.78713330e-01],
       [1.41199080e-02, 9.85877040e-01, 3.00245780e-06],
       [2.47453380e-01, 7.43928550e-01, 8.61808000e-03],
       [2.02928800e-03, 3.38499970e-03, 9.94585700e-01],
       [8.32148300e-03, 1.29651830e-02, 9.78713330e-01],
       [2.02928800e-03, 3.38499970e-03, 9.94585700e-01],
       [9.70933500e-01, 2.89793950e-02, 8.71625740e-05],
       [9.59613800e-01, 3.77857760e-02, 2.60037100e-03],
       [6.21423660e-01, 3.73048660e-01, 5.52768870e-03],
       [1.03151050e-01, 8.91719040e-01, 5.12992000e-03],
       [2.02928800e-03, 3.38499970e-03, 9.94585700e-01],
       [2.02928800e-03, 3.38499970e-03, 9.94585700e-01],
       [8.32148300e-03, 1.29651

In [53]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9877712031558185

In [54]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9877712031558185

In [55]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9844773175542406

In [56]:
np.std(ovos)

0.007061832633107952

In [57]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9844773175542406

In [58]:
np.std(ovrs)

0.007061832633107952

In [103]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [104]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 93.85%


In [105]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.012561485111703652


In [106]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [107]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 96.70%


In [108]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.0079398835


In [59]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_over)

In [60]:
dat5 = pd.DataFrame(X_test_over[:,0])
dat5['test'] = y_test_over

In [61]:
dat5

,0,test
0,312,1
1,BCH-SA-12,0
2,NRS209,2
3,CFBRSa29,0
4,NRS209,2
...,...,...
190,CA541,1
191,SR4152,1
192,NRS110,2
193,CFBRSa70,0


In [62]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [63]:
#### add regularizer and dropout
model1_over5 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [97]:
model1_over5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 467us/step - loss: 8.4312 - accuracy: 0.4702 - val_loss: 5.8295 - val_accuracy: 0.5333
Epoch 2/100
453/453 [==============================] - 0s 111us/step - loss: 7.9428 - accuracy: 0.4724 - val_loss: 5.4704 - val_accuracy: 0.5436
Epoch 3/100
453/453 [==============================] - 0s 125us/step - loss: 7.2336 - accuracy: 0.5188 - val_loss: 5.2938 - val_accuracy: 0.5692
Epoch 4/100
453/453 [==============================] - 0s 136us/step - loss: 7.1452 - accuracy: 0.5099 - val_loss: 4.9137 - val_accuracy: 0.5692
Epoch 5/100
453/453 [==============================] - 0s 138us/step - loss: 6.4976 - accuracy: 0.5210 - val_loss: 2.8596 - val_accuracy: 0.6000
Epoch 6/100
453/453 [==============================] - 0s 118us/step - loss: 4.1453 - accuracy: 0.6291 - val_loss: 1.8736 - val_accuracy: 0.6872
Epoch 7/100
453/453 [==============================] - 0s 103us/step - loss: 4.1718 

453/453 [==============================] - 0s 98us/step - loss: 2.3843 - accuracy: 0.7616 - val_loss: 0.9301 - val_accuracy: 0.8667
Epoch 58/100
453/453 [==============================] - 0s 178us/step - loss: 2.5630 - accuracy: 0.7395 - val_loss: 0.9181 - val_accuracy: 0.8615
Epoch 59/100
453/453 [==============================] - 0s 164us/step - loss: 2.0944 - accuracy: 0.7682 - val_loss: 0.9390 - val_accuracy: 0.8513
Epoch 60/100
453/453 [==============================] - 0s 197us/step - loss: 2.1779 - accuracy: 0.7285 - val_loss: 0.9153 - val_accuracy: 0.8564
Epoch 61/100
453/453 [==============================] - 0s 147us/step - loss: 2.2257 - accuracy: 0.7572 - val_loss: 0.9119 - val_accuracy: 0.8564
Epoch 62/100
453/453 [==============================] - 0s 161us/step - loss: 2.0256 - accuracy: 0.7483 - val_loss: 0.9589 - val_accuracy: 0.8205
Epoch 63/100
453/453 [==============================] - 0s 131us/step - loss: 2.1729 - accuracy: 0.7174 - val_loss: 0.9244 - val_accuracy:

In [124]:
acc_test_over5 = model1_over5.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over5*100))

195/195 [==============================] - 0s 63us/step
over-sampling test accuracy: 90.77%


In [99]:
pred5 = model1_over5.predict_classes(X_test_over)
pred5

array([1, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 0, 0, 2, 2, 2,
       1, 2, 2, 1, 0, 1, 2, 0, 1, 0, 2, 2, 1, 1, 2, 2, 2, 0, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 2, 0, 2, 1, 0, 2, 0, 0, 0, 2, 1, 1, 0, 2, 0, 0, 0,
       2, 2, 1, 1, 0, 0, 1, 1, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 2, 2, 0, 0, 0, 0,
       2, 1, 2, 1, 1, 0, 0, 1, 1, 1, 1, 2, 0, 1, 0, 2, 2, 0, 0, 0, 2, 1,
       2, 0, 1, 0, 2, 1, 1, 2, 2, 0, 2, 1, 0, 0, 0, 2, 2, 2, 1, 1, 2, 2,
       0, 0, 1, 0, 2, 1, 0, 1, 2, 2, 0, 0, 2, 2, 2, 2, 1, 1, 0, 1, 0, 1,
       2, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 1, 2, 0, 1])

In [100]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,312,1,1
1,BCH-SA-12,0,0
2,NRS209,2,2
3,CFBRSa29,0,0
4,NRS209,2,2
...,...,...,...
190,CA541,1,0
191,SR4152,1,1
192,NRS110,2,2
193,CFBRSa70,0,0


In [101]:
proba5 = model1_over5.predict_proba(X_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [102]:
dat_proba5

,0,1,2
0,4.310851e-01,5.689149e-01,7.807134e-08
1,9.999999e-01,8.498781e-08,1.082131e-10
2,1.343175e-08,9.256006e-08,9.999999e-01
3,1.000000e+00,8.183677e-09,1.139258e-10
4,1.343175e-08,9.256006e-08,9.999999e-01
...,...,...,...
190,7.755124e-01,2.244875e-01,1.170079e-07
191,3.495487e-02,9.650442e-01,8.668809e-07
192,4.195798e-08,3.755047e-08,9.999999e-01
193,8.460521e-01,1.539446e-01,3.216454e-06


In [103]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [104]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p17spST.csv", index = False,
         header=None)

In [128]:
hist1_over5 = model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 142us/step - loss: 1.7036 - accuracy: 0.7616 - val_loss: 0.7411 - val_accuracy: 0.9231
Epoch 2/100
453/453 [==============================] - 0s 166us/step - loss: 1.8317 - accuracy: 0.6932 - val_loss: 0.7692 - val_accuracy: 0.9128
Epoch 3/100
453/453 [==============================] - 0s 143us/step - loss: 1.8548 - accuracy: 0.7086 - val_loss: 0.9420 - val_accuracy: 0.8974
Epoch 4/100
453/453 [==============================] - 0s 136us/step - loss: 1.9690 - accuracy: 0.7373 - val_loss: 0.8254 - val_accuracy: 0.8564
Epoch 5/100
453/453 [==============================] - 0s 128us/step - loss: 1.5297 - accuracy: 0.7748 - val_loss: 0.8122 - val_accuracy: 0.8564
Epoch 6/100
453/453 [==============================] - 0s 133us/step - loss: 1.6204 - accuracy: 0.7351 - val_loss: 0.8708 - val_accuracy: 0.8615
Epoch 7/100
453/453 [==============================] - 0s 142us/step - loss: 1.5643 

Epoch 57/100
453/453 [==============================] - 0s 155us/step - loss: 1.6184 - accuracy: 0.7550 - val_loss: 0.6611 - val_accuracy: 0.9282
Epoch 58/100
453/453 [==============================] - 0s 114us/step - loss: 1.6447 - accuracy: 0.7439 - val_loss: 0.7121 - val_accuracy: 0.9282
Epoch 59/100
453/453 [==============================] - 0s 143us/step - loss: 1.3576 - accuracy: 0.7594 - val_loss: 0.6996 - val_accuracy: 0.8923
Epoch 60/100
453/453 [==============================] - 0s 152us/step - loss: 1.6312 - accuracy: 0.7572 - val_loss: 0.6569 - val_accuracy: 0.9282
Epoch 61/100
453/453 [==============================] - 0s 105us/step - loss: 1.3088 - accuracy: 0.7770 - val_loss: 0.7766 - val_accuracy: 0.9128
Epoch 62/100
453/453 [==============================] - 0s 111us/step - loss: 1.5837 - accuracy: 0.7351 - val_loss: 0.6797 - val_accuracy: 0.9282
Epoch 63/100
453/453 [==============================] - 0s 243us/step - loss: 1.6278 - accuracy: 0.6954 - val_loss: 0.6495 -

In [129]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over5.history['accuracy'])*100))

over-sampling train accuracy: 74.90%


In [64]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [65]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS241,1,1,1.342914e-03,9.986569e-01,2.348628e-07
1,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
2,p002ykpresabsSTCC_qual,NRS255,1,1,1.780311e-07,9.999999e-01,2.544841e-12
3,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,2.203547e-10,5.688883e-15
4,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,BCH-SA-12,0,0,1.000000e+00,1.152503e-09,1.898730e-09
1978,pyopresabsSTCC_qual,NRS049,0,1,8.401357e-11,1.000000e+00,3.209735e-13
1979,pyopresabsSTCC_qual,NRS022,0,0,1.000000e+00,4.755084e-10,1.974275e-10
1980,pyopresabsSTCC_qual,NRS236,1,1,1.357345e-08,1.000000e+00,1.293117e-10


In [66]:
y_prob5 = df_proba5[df_proba5['phage']=='p0017SpresabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[4.31085050e-01, 5.68914950e-01, 7.80713400e-08],
       [9.99999900e-01, 8.49878100e-08, 1.08213105e-10],
       [1.34317460e-08, 9.25600650e-08, 9.99999900e-01],
       [1.00000000e+00, 8.18367700e-09, 1.13925834e-10],
       [1.34317460e-08, 9.25600650e-08, 9.99999900e-01],
       [9.99975700e-01, 2.43147060e-05, 1.01738140e-10],
       [1.00000000e+00, 3.56445800e-10, 1.96984100e-13],
       [4.19579820e-08, 3.75504680e-08, 9.99999900e-01],
       [9.99899600e-01, 1.00377370e-04, 1.12083946e-10],
       [1.34317460e-08, 9.25600650e-08, 9.99999900e-01],
       [4.19579820e-08, 3.75504680e-08, 9.99999900e-01],
       [4.19579820e-08, 3.75504680e-08, 9.99999900e-01],
       [4.19579820e-08, 3.75504680e-08, 9.99999900e-01],
       [4.19579820e-08, 3.75504680e-08, 9.99999900e-01],
       [1.74528570e-02, 9.82547000e-01, 6.79716200e-08],
       [4.19579820e-08, 3.75504680e-08, 9.99999900e-01],
       [1.34317460e-08, 9.25600650e-08, 9.99999900e-01],
       [8.60922300e-01, 1.38499

In [67]:
ovo5 = rocauc_ovo(y_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.9714792899408283

In [68]:
ovr5 = rocauc_ovr(y_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.9714792899408283

In [69]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_over)

In [70]:
dat6 = pd.DataFrame(X_test_over[:,0])
dat6['test'] = y_test_over

In [71]:
dat6

,0,test
0,CFBREBSa119,0
1,NRS001,1
2,NRS074,0
3,NRS209,2
4,GA231,0
...,...,...
190,NRS252,0
191,SR2852,1
192,NRS108,1
193,NRS202,0


In [72]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [73]:
model1_over6 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [110]:
model1_over6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 581us/step - loss: 9.6541 - accuracy: 0.3907 - val_loss: 6.0553 - val_accuracy: 0.3949
Epoch 2/100
453/453 [==============================] - 0s 139us/step - loss: 8.0938 - accuracy: 0.4283 - val_loss: 4.5736 - val_accuracy: 0.4256
Epoch 3/100
453/453 [==============================] - 0s 140us/step - loss: 7.2743 - accuracy: 0.5386 - val_loss: 3.2626 - val_accuracy: 0.6923
Epoch 4/100
453/453 [==============================] - 0s 140us/step - loss: 6.1172 - accuracy: 0.5188 - val_loss: 2.7606 - val_accuracy: 0.6000
Epoch 5/100
453/453 [==============================] - 0s 111us/step - loss: 5.6745 - accuracy: 0.5673 - val_loss: 2.3318 - val_accuracy: 0.6513
Epoch 6/100
453/453 [==============================] - 0s 130us/step - loss: 5.3711 - accuracy: 0.5762 - val_loss: 1.9673 - val_accuracy: 0.7846
Epoch 7/100
453/453 [==============================] - 0s 159us/step - loss: 4.7168 

Epoch 57/100
453/453 [==============================] - 0s 102us/step - loss: 2.0916 - accuracy: 0.7152 - val_loss: 0.8274 - val_accuracy: 0.8667
Epoch 58/100
453/453 [==============================] - 0s 93us/step - loss: 2.0693 - accuracy: 0.7616 - val_loss: 0.7692 - val_accuracy: 0.8667
Epoch 59/100
453/453 [==============================] - 0s 153us/step - loss: 2.0332 - accuracy: 0.7351 - val_loss: 0.7520 - val_accuracy: 0.8564
Epoch 60/100
453/453 [==============================] - 0s 130us/step - loss: 1.9196 - accuracy: 0.7174 - val_loss: 0.7642 - val_accuracy: 0.8667
Epoch 61/100
453/453 [==============================] - 0s 138us/step - loss: 1.8954 - accuracy: 0.7770 - val_loss: 0.7262 - val_accuracy: 0.8769
Epoch 62/100
453/453 [==============================] - 0s 148us/step - loss: 2.0452 - accuracy: 0.7351 - val_loss: 0.7382 - val_accuracy: 0.8564
Epoch 63/100
453/453 [==============================] - 0s 138us/step - loss: 2.0040 - accuracy: 0.7572 - val_loss: 0.7201 - 

In [145]:
acc_test_over6 = model1_over6.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over6*100))

195/195 [==============================] - 0s 70us/step
over-sampling test accuracy: 90.26%


In [112]:
pred6 = model1_over6.predict_classes(X_test_over)
pred6

array([0, 1, 0, 2, 0, 2, 0, 0, 1, 1, 2, 1, 2, 0, 0, 0, 2, 2, 2, 0, 2, 1,
       0, 1, 1, 0, 0, 1, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 1, 0, 1, 1,
       2, 2, 2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 2, 0, 2, 1, 1, 0, 0, 1,
       1, 2, 2, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 2, 0, 0, 0, 1, 0,
       1, 2, 1, 1, 2, 2, 1, 1, 0, 2, 0, 2, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2,
       2, 1, 0, 1, 2, 0, 0, 0, 2, 1, 2, 1, 1, 2, 2, 0, 0, 2, 2, 0, 1, 0,
       1, 1, 1, 2, 1, 2, 1, 0, 2, 0, 2, 0, 2, 1, 2, 1, 0, 2, 0, 2, 2, 2,
       2, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 1, 0, 0, 1, 0, 1, 2, 0, 0, 1, 1,
       0, 2, 2, 2, 1, 0, 2, 1, 1, 2, 2, 1, 2, 1, 0, 1, 1, 0, 2])

In [113]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,CFBREBSa119,0,0
1,NRS001,1,1
2,NRS074,0,0
3,NRS209,2,2
4,GA231,0,0
...,...,...,...
190,NRS252,0,0
191,SR2852,1,1
192,NRS108,1,1
193,NRS202,0,0


In [114]:
proba6 = model1_over6.predict_proba(X_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [115]:
dat_proba6

,0,1,2
0,5.547507e-01,4.449964e-01,2.528183e-04
1,1.769752e-01,8.229654e-01,5.931539e-05
2,1.000000e+00,1.297128e-08,2.716962e-10
3,5.973688e-08,5.079579e-08,9.999999e-01
4,5.744864e-01,4.255115e-01,2.071251e-06
...,...,...,...
190,7.873229e-01,2.126577e-01,1.932460e-05
191,1.636022e-04,9.998363e-01,1.226338e-09
192,2.683811e-08,1.000000e+00,4.172592e-11
193,7.581264e-01,2.402929e-01,1.580697e-03


In [116]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [117]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p17spST.csv", index = False,
         header=None)

In [149]:
hist1_over6 = model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 221us/step - loss: 1.9247 - accuracy: 0.7241 - val_loss: 0.7858 - val_accuracy: 0.8667
Epoch 2/100
453/453 [==============================] - 0s 142us/step - loss: 1.8978 - accuracy: 0.7528 - val_loss: 0.7794 - val_accuracy: 0.8667
Epoch 3/100
453/453 [==============================] - 0s 121us/step - loss: 1.6541 - accuracy: 0.7572 - val_loss: 0.7422 - val_accuracy: 0.8769
Epoch 4/100
453/453 [==============================] - 0s 136us/step - loss: 1.7294 - accuracy: 0.7550 - val_loss: 0.7210 - val_accuracy: 0.8769
Epoch 5/100
453/453 [==============================] - 0s 160us/step - loss: 1.8397 - accuracy: 0.7770 - val_loss: 0.7604 - val_accuracy: 0.8718
Epoch 6/100
453/453 [==============================] - 0s 128us/step - loss: 1.8789 - accuracy: 0.6976 - val_loss: 0.7550 - val_accuracy: 0.8821
Epoch 7/100
453/453 [==============================] - 0s 144us/step - loss: 1.5993 

Epoch 57/100
453/453 [==============================] - 0s 163us/step - loss: 1.3797 - accuracy: 0.7815 - val_loss: 0.8279 - val_accuracy: 0.8872
Epoch 58/100
453/453 [==============================] - 0s 173us/step - loss: 1.7186 - accuracy: 0.7726 - val_loss: 0.7775 - val_accuracy: 0.8974
Epoch 59/100
453/453 [==============================] - 0s 143us/step - loss: 1.6284 - accuracy: 0.7373 - val_loss: 0.7934 - val_accuracy: 0.8821
Epoch 60/100
453/453 [==============================] - 0s 140us/step - loss: 1.5067 - accuracy: 0.7660 - val_loss: 0.8021 - val_accuracy: 0.8974
Epoch 61/100
453/453 [==============================] - 0s 149us/step - loss: 1.4870 - accuracy: 0.7726 - val_loss: 0.8677 - val_accuracy: 0.9077
Epoch 62/100
453/453 [==============================] - 0s 158us/step - loss: 1.3330 - accuracy: 0.7859 - val_loss: 0.8025 - val_accuracy: 0.8923
Epoch 63/100
453/453 [==============================] - 0s 128us/step - loss: 1.4671 - accuracy: 0.7770 - val_loss: 0.8261 -

In [150]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over6.history['accuracy'])*100))

over-sampling train accuracy: 76.11%


In [74]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [75]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,1.790400e-08,4.141849e-08,1.000000e+00
1,p002ykpresabsSTCC_qual,NRS386,1,1,5.739934e-04,9.994259e-01,6.773014e-08
2,p002ykpresabsSTCC_qual,NRS148,2,2,5.286934e-09,1.269109e-08,1.000000e+00
3,p002ykpresabsSTCC_qual,NRS178,0,1,6.494936e-12,1.000000e+00,2.537080e-25
4,p002ykpresabsSTCC_qual,NRS237,0,1,5.701098e-02,9.399204e-01,3.068583e-03
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS272,0,0,9.999607e-01,3.367024e-05,5.776848e-06
1978,pyopresabsSTCC_qual,NRS112,1,1,8.275442e-08,9.999999e-01,3.739556e-09
1979,pyopresabsSTCC_qual,NRS064,1,1,2.168245e-08,1.000000e+00,9.603962e-09
1980,pyopresabsSTCC_qual,BCH-SA-04,0,0,1.000000e+00,1.026408e-15,1.630406e-14


In [77]:
y_prob6 = df_proba6[df_proba6['phage']=='p0017SpresabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[5.54750740e-01, 4.44996420e-01, 2.52818300e-04],
       [1.76975160e-01, 8.22965440e-01, 5.93153850e-05],
       [1.00000000e+00, 1.29712800e-08, 2.71696170e-10],
       [5.97368840e-08, 5.07957940e-08, 9.99999900e-01],
       [5.74486430e-01, 4.25511540e-01, 2.07125070e-06],
       [7.67440700e-08, 7.78947960e-08, 9.99999900e-01],
       [1.00000000e+00, 8.52028300e-09, 1.62860990e-10],
       [9.51851100e-01, 4.75664330e-02, 5.82489500e-04],
       [9.97858940e-02, 8.97806760e-01, 2.40733520e-03],
       [1.49431210e-01, 8.50565850e-01, 2.95004000e-06],
       [5.97368840e-08, 5.07957940e-08, 9.99999900e-01],
       [3.12902630e-02, 9.68696650e-01, 1.30889200e-05],
       [7.67440700e-08, 7.78947960e-08, 9.99999900e-01],
       [9.98425960e-01, 1.57388200e-03, 2.47838840e-07],
       [9.97186360e-01, 2.81309150e-03, 5.03539070e-07],
       [9.86041700e-01, 1.39416065e-02, 1.66439910e-05],
       [7.67440700e-08, 7.78947960e-08, 9.99999900e-01],
       [5.97368840e-08, 5.07957

In [78]:
ovo6 = rocauc_ovo(y_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.9694280078895464

In [79]:
ovr6 = rocauc_ovr(y_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.9694280078895464

In [80]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_over)

In [81]:
dat7 = pd.DataFrame(X_test_over[:,0])
dat7['test'] = y_test_over

In [82]:
dat7

,0,test
0,NRS218,1
1,NRS260,1
2,NRS162,0
3,NRS177,0
4,NRS209,2
...,...,...
190,NRS383,1
191,NRS218,1
192,NRS209,2
193,SR2852,1


In [83]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [84]:
model1_over7 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [123]:
model1_over7.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [124]:
model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 590us/step - loss: 11.7977 - accuracy: 0.3974 - val_loss: 10.8170 - val_accuracy: 0.4410
Epoch 2/100
453/453 [==============================] - 0s 133us/step - loss: 10.1523 - accuracy: 0.5055 - val_loss: 9.6470 - val_accuracy: 0.4923
Epoch 3/100
453/453 [==============================] - 0s 91us/step - loss: 9.6769 - accuracy: 0.4879 - val_loss: 8.6653 - val_accuracy: 0.4923
Epoch 4/100
453/453 [==============================] - 0s 106us/step - loss: 8.1876 - accuracy: 0.5342 - val_loss: 7.8644 - val_accuracy: 0.5333
Epoch 5/100
453/453 [==============================] - 0s 94us/step - loss: 7.6728 - accuracy: 0.5232 - val_loss: 7.1779 - val_accuracy: 0.5385
Epoch 6/100
453/453 [==============================] - 0s 153us/step - loss: 7.6673 - accuracy: 0.4923 - val_loss: 6.5479 - val_accuracy: 0.5128
Epoch 7/100
453/453 [==============================] - 0s 197us/step - loss: 6.7987

Epoch 57/100
453/453 [==============================] - 0s 124us/step - loss: 2.2738 - accuracy: 0.7373 - val_loss: 0.7796 - val_accuracy: 0.8667
Epoch 58/100
453/453 [==============================] - 0s 128us/step - loss: 2.1465 - accuracy: 0.7417 - val_loss: 0.7009 - val_accuracy: 0.8615
Epoch 59/100
453/453 [==============================] - 0s 94us/step - loss: 1.9698 - accuracy: 0.7616 - val_loss: 0.7442 - val_accuracy: 0.8615
Epoch 60/100
453/453 [==============================] - 0s 115us/step - loss: 2.4382 - accuracy: 0.7241 - val_loss: 0.8284 - val_accuracy: 0.8615
Epoch 61/100
453/453 [==============================] - 0s 119us/step - loss: 1.9757 - accuracy: 0.7528 - val_loss: 0.9702 - val_accuracy: 0.8564
Epoch 62/100
453/453 [==============================] - 0s 118us/step - loss: 2.2523 - accuracy: 0.7285 - val_loss: 1.0378 - val_accuracy: 0.8718
Epoch 63/100
453/453 [==============================] - 0s 119us/step - loss: 2.5103 - accuracy: 0.7064 - val_loss: 0.8574 - 

In [158]:
acc_test_over7 = model1_over7.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over7*100))

195/195 [==============================] - 0s 94us/step
over-sampling test accuracy: 89.74%


In [125]:
pred7 = model1_over7.predict_classes(X_test_over)
pred7

array([1, 1, 0, 0, 2, 0, 1, 0, 2, 2, 1, 2, 0, 1, 0, 1, 0, 0, 1, 1, 2, 2,
       2, 2, 1, 1, 1, 1, 1, 0, 1, 0, 0, 2, 2, 1, 1, 0, 2, 0, 2, 0, 2, 0,
       0, 0, 1, 2, 1, 2, 2, 0, 0, 0, 0, 2, 2, 1, 1, 1, 1, 0, 2, 2, 0, 1,
       2, 1, 0, 2, 2, 0, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 0,
       0, 2, 1, 2, 2, 1, 2, 1, 0, 1, 0, 2, 2, 2, 0, 0, 1, 0, 2, 0, 0, 1,
       0, 1, 2, 0, 2, 1, 0, 1, 0, 1, 0, 1, 2, 0, 1, 2, 0, 0, 0, 2, 1, 1,
       2, 1, 1, 1, 1, 0, 1, 0, 0, 2, 2, 1, 0, 2, 1, 0, 1, 2, 2, 2, 0, 0,
       2, 1, 2, 2, 1, 2, 2, 0, 0, 0, 2, 0, 0, 2, 1, 1, 1, 0, 0, 0, 2, 1,
       1, 0, 0, 1, 2, 1, 1, 0, 2, 0, 2, 1, 0, 2, 0, 1, 2, 1, 0])

In [126]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS218,1,1
1,NRS260,1,1
2,NRS162,0,0
3,NRS177,0,0
4,NRS209,2,2
...,...,...,...
190,NRS383,1,0
191,NRS218,1,1
192,NRS209,2,2
193,SR2852,1,1


In [127]:
proba7 = model1_over7.predict_proba(X_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [128]:
dat_proba7

,0,1,2
0,3.540596e-02,9.518660e-01,1.272812e-02
1,5.710814e-03,9.939436e-01,3.454555e-04
2,9.999807e-01,1.819183e-05,1.128070e-06
3,9.999942e-01,5.873688e-06,1.826433e-08
4,7.429018e-08,6.771530e-08,9.999999e-01
...,...,...,...
190,8.690237e-01,1.309742e-01,2.209219e-06
191,3.540596e-02,9.518660e-01,1.272812e-02
192,7.429018e-08,6.771530e-08,9.999999e-01
193,3.114946e-03,9.968849e-01,6.330789e-08


In [129]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [130]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p17spST.csv", index = False,
         header=None)

In [162]:
hist1_over7 = model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 129us/step - loss: 1.9407 - accuracy: 0.7373 - val_loss: 0.7832 - val_accuracy: 0.9077
Epoch 2/100
453/453 [==============================] - 0s 127us/step - loss: 1.6920 - accuracy: 0.7285 - val_loss: 0.8520 - val_accuracy: 0.8923
Epoch 3/100
453/453 [==============================] - 0s 157us/step - loss: 1.7241 - accuracy: 0.7351 - val_loss: 0.8375 - val_accuracy: 0.9026
Epoch 4/100
453/453 [==============================] - 0s 118us/step - loss: 1.9081 - accuracy: 0.7528 - val_loss: 0.8226 - val_accuracy: 0.9128
Epoch 5/100
453/453 [==============================] - 0s 106us/step - loss: 1.8447 - accuracy: 0.7219 - val_loss: 0.9478 - val_accuracy: 0.8667
Epoch 6/100
453/453 [==============================] - 0s 107us/step - loss: 1.7675 - accuracy: 0.7196 - val_loss: 0.8372 - val_accuracy: 0.8718
Epoch 7/100
453/453 [==============================] - 0s 118us/step - loss: 1.6275 

In [163]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over7.history['accuracy'])*100))

over-sampling train accuracy: 75.55%


In [85]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [86]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
1,p002ykpresabsSTCC_qual,BCH-SA-09,1,1,1.137139e-06,9.999988e-01,2.067601e-09
2,p002ykpresabsSTCC_qual,NRS224,0,0,1.000000e+00,2.093110e-31,0.000000e+00
3,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
4,p002ykpresabsSTCC_qual,NRS235,1,1,2.243513e-02,9.774035e-01,1.615106e-04
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS035,0,0,9.354528e-01,6.414209e-02,4.051121e-04
1978,pyopresabsSTCC_qual,NRS260,1,1,4.808470e-08,1.000000e+00,7.364639e-09
1979,pyopresabsSTCC_qual,CA9,0,0,1.000000e+00,2.361323e-08,2.871247e-08
1980,pyopresabsSTCC_qual,NRS183,1,1,2.755864e-07,9.999998e-01,5.310879e-08


In [87]:
y_prob7 = df_proba7[df_proba7['phage']=='p0017SpresabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[3.54059560e-02, 9.51866000e-01, 1.27281210e-02],
       [5.71081400e-03, 9.93943630e-01, 3.45455480e-04],
       [9.99980700e-01, 1.81918350e-05, 1.12807050e-06],
       [9.99994160e-01, 5.87368800e-06, 1.82643340e-08],
       [7.42901800e-08, 6.77153000e-08, 9.99999900e-01],
       [1.00000000e+00, 3.11236180e-11, 5.54551200e-13],
       [5.09788400e-02, 9.49020560e-01, 5.65980100e-07],
       [8.75602000e-01, 1.24389924e-01, 8.00534050e-06],
       [7.42901800e-08, 6.77153000e-08, 9.99999900e-01],
       [7.42901800e-08, 6.77153000e-08, 9.99999900e-01],
       [6.68401300e-02, 9.33159900e-01, 2.40770050e-08],
       [7.89520200e-08, 7.79008900e-08, 9.99999900e-01],
       [8.12116440e-01, 1.87881800e-01, 1.78686130e-06],
       [8.33406740e-05, 9.99916550e-01, 7.52781200e-08],
       [8.93029450e-01, 1.06964940e-01, 5.61495060e-06],
       [1.27348400e-05, 9.99982600e-01, 4.63852170e-06],
       [9.80462970e-01, 1.95370610e-02, 7.84910800e-10],
       [1.00000000e+00, 3.28920

In [88]:
ovo7 = rocauc_ovo(y_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.968836291913215

In [89]:
ovr7 = rocauc_ovr(y_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.968836291913215

In [90]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_over)

In [91]:
dat8 = pd.DataFrame(X_test_over[:,0])
dat8['test'] = y_test_over

In [92]:
dat8

,0,test
0,NRS209,2
1,NRS255,1
2,NRS119,0
3,NRS071,0
4,NRS002,0
...,...,...
190,CFBRSa30,0
191,NRS383,1
192,NRS110,2
193,NRS209,2


In [93]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [94]:
model1_over8 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [136]:
model1_over8.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [137]:
model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 634us/step - loss: 9.3642 - accuracy: 0.3709 - val_loss: 8.2229 - val_accuracy: 0.5077
Epoch 2/100
453/453 [==============================] - 0s 106us/step - loss: 7.8240 - accuracy: 0.4746 - val_loss: 6.6678 - val_accuracy: 0.5385
Epoch 3/100
453/453 [==============================] - 0s 155us/step - loss: 6.8921 - accuracy: 0.5033 - val_loss: 5.5176 - val_accuracy: 0.5692
Epoch 4/100
453/453 [==============================] - 0s 158us/step - loss: 5.9287 - accuracy: 0.5563 - val_loss: 4.2639 - val_accuracy: 0.6359
Epoch 5/100
453/453 [==============================] - 0s 182us/step - loss: 5.4087 - accuracy: 0.6137 - val_loss: 2.8019 - val_accuracy: 0.7487
Epoch 6/100
453/453 [==============================] - 0s 252us/step - loss: 4.8944 - accuracy: 0.6247 - val_loss: 2.0542 - val_accuracy: 0.6872
Epoch 7/100
453/453 [==============================] - 0s 177us/step - loss: 4.2195 

453/453 [==============================] - 0s 186us/step - loss: 2.7676 - accuracy: 0.6843 - val_loss: 0.6913 - val_accuracy: 0.8769
Epoch 58/100
453/453 [==============================] - 0s 150us/step - loss: 2.4750 - accuracy: 0.7241 - val_loss: 0.6360 - val_accuracy: 0.8974
Epoch 59/100
453/453 [==============================] - 0s 97us/step - loss: 2.0600 - accuracy: 0.7660 - val_loss: 0.6150 - val_accuracy: 0.8974
Epoch 60/100
453/453 [==============================] - 0s 122us/step - loss: 2.4241 - accuracy: 0.7395 - val_loss: 0.5947 - val_accuracy: 0.9231
Epoch 61/100
453/453 [==============================] - 0s 200us/step - loss: 2.1549 - accuracy: 0.7373 - val_loss: 0.5797 - val_accuracy: 0.9385
Epoch 62/100
453/453 [==============================] - 0s 209us/step - loss: 2.2881 - accuracy: 0.7572 - val_loss: 0.6496 - val_accuracy: 0.8923
Epoch 63/100
453/453 [==============================] - 0s 159us/step - loss: 2.2814 - accuracy: 0.7285 - val_loss: 0.6258 - val_accuracy:

In [171]:
acc_test_over8 = model1_over8.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over8*100))

195/195 [==============================] - 0s 51us/step
over-sampling test accuracy: 91.79%


In [138]:
pred8 = model1_over8.predict_classes(X_test_over)
pred8

array([2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 2, 1, 0, 0, 2, 2, 1, 0, 1, 2,
       1, 0, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 1, 2, 0,
       2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 2, 1,
       0, 0, 1, 2, 0, 1, 2, 0, 1, 1, 0, 1, 2, 0, 1, 2, 2, 2, 1, 0, 1, 2,
       2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 0, 0, 1, 1, 0, 1, 2, 2, 2, 1, 0,
       0, 2, 1, 0, 0, 2, 1, 0, 2, 1, 0, 1, 2, 0, 1, 2, 1, 1, 0, 0, 0, 2,
       2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 2, 2, 0, 1, 2, 0, 1, 2, 2, 0, 2, 1,
       0, 0, 1, 2, 1, 1, 0, 2, 2, 1, 1, 2, 2, 2, 1, 1, 0, 0, 2, 0, 2, 2,
       1, 2, 1, 1, 0, 2, 1, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 0])

In [139]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,NRS209,2,2
1,NRS255,1,1
2,NRS119,0,0
3,NRS071,0,0
4,NRS002,0,1
...,...,...,...
190,CFBRSa30,0,0
191,NRS383,1,1
192,NRS110,2,2
193,NRS209,2,2


In [140]:
proba8 = model1_over8.predict_proba(X_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [141]:
dat_proba8

,0,1,2
0,1.839877e-06,4.253379e-07,9.999977e-01
1,4.268283e-07,9.999995e-01,3.068627e-08
2,9.579664e-01,4.170085e-02,3.327874e-04
3,9.974597e-01,2.538614e-03,1.713039e-06
4,2.087385e-02,9.791145e-01,1.152091e-05
...,...,...,...
190,9.999975e-01,2.553651e-06,1.778455e-09
191,6.077641e-02,9.384032e-01,8.203266e-04
192,7.736103e-08,4.280218e-08,9.999999e-01
193,1.839877e-06,4.253379e-07,9.999977e-01


In [142]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [143]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p17spST.csv", index = False,
         header=None)

In [175]:
hist1_over8 = model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/100
453/453 [==============================] - 0s 129us/step - loss: 1.7257 - accuracy: 0.7594 - val_loss: 0.8542 - val_accuracy: 0.8872
Epoch 2/100
453/453 [==============================] - 0s 121us/step - loss: 1.9077 - accuracy: 0.7660 - val_loss: 0.8119 - val_accuracy: 0.8821
Epoch 3/100
453/453 [==============================] - 0s 114us/step - loss: 1.9186 - accuracy: 0.7196 - val_loss: 0.7299 - val_accuracy: 0.8821
Epoch 4/100
453/453 [==============================] - 0s 104us/step - loss: 1.8625 - accuracy: 0.7064 - val_loss: 0.6569 - val_accuracy: 0.9128
Epoch 5/100
453/453 [==============================] - 0s 104us/step - loss: 1.7785 - accuracy: 0.7616 - val_loss: 0.7111 - val_accuracy: 0.9128
Epoch 6/100
453/453 [==============================] - 0s 123us/step - loss: 1.8470 - accuracy: 0.7726 - val_loss: 0.7371 - val_accuracy: 0.9179
Epoch 7/100
453/453 [==============================] - 0s 107us/step - loss: 1.5629 

Epoch 57/100
453/453 [==============================] - 0s 105us/step - loss: 1.2419 - accuracy: 0.7572 - val_loss: 0.5781 - val_accuracy: 0.9128
Epoch 58/100
453/453 [==============================] - 0s 99us/step - loss: 1.3725 - accuracy: 0.7572 - val_loss: 0.5665 - val_accuracy: 0.9026
Epoch 59/100
453/453 [==============================] - 0s 108us/step - loss: 1.5217 - accuracy: 0.7483 - val_loss: 0.5750 - val_accuracy: 0.9128
Epoch 60/100
453/453 [==============================] - 0s 121us/step - loss: 1.2561 - accuracy: 0.7660 - val_loss: 0.5933 - val_accuracy: 0.9333
Epoch 61/100
453/453 [==============================] - 0s 110us/step - loss: 1.6316 - accuracy: 0.7395 - val_loss: 0.6002 - val_accuracy: 0.9179
Epoch 62/100
453/453 [==============================] - 0s 87us/step - loss: 1.6017 - accuracy: 0.7373 - val_loss: 0.5768 - val_accuracy: 0.9026
Epoch 63/100
453/453 [==============================] - 0s 93us/step - loss: 1.4606 - accuracy: 0.7528 - val_loss: 0.6490 - va

In [176]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over8.history['accuracy'])*100))

over-sampling train accuracy: 75.32%


In [95]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [96]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,CFBREBSa116,0,0,9.676203e-01,3.237956e-02,1.480166e-07
1,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,6.534852e-11,2.250731e-18
2,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
3,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
4,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS205,2,2,3.691095e-08,3.571927e-08,9.999999e-01
1978,pyopresabsSTCC_qual,CFBREBSa122,0,1,9.261665e-02,9.073822e-01,1.162373e-06
1979,pyopresabsSTCC_qual,NRS001,1,1,4.174278e-07,9.999995e-01,3.254024e-09
1980,pyopresabsSTCC_qual,NRS148,2,2,3.234670e-08,3.121212e-08,9.999999e-01


In [97]:
y_prob8 = df_proba8[df_proba8['phage']=='p0017SpresabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[1.83987700e-06, 4.25337900e-07, 9.99997740e-01],
       [4.26828340e-07, 9.99999500e-01, 3.06862730e-08],
       [9.57966400e-01, 4.17008500e-02, 3.32787430e-04],
       [9.97459700e-01, 2.53861350e-03, 1.71303860e-06],
       [2.08738500e-02, 9.79114530e-01, 1.15209150e-05],
       [1.00000000e+00, 1.30497710e-08, 3.26204880e-12],
       [1.00000000e+00, 2.24513270e-09, 7.68191900e-12],
       [9.99999640e-01, 3.26373030e-07, 3.59689880e-09],
       [9.81602900e-03, 9.90170660e-01, 1.33881870e-05],
       [9.99949700e-01, 5.03017200e-05, 3.76725000e-10],
       [7.35023200e-02, 9.26497460e-01, 2.03305620e-07],
       [1.00000000e+00, 2.80718800e-10, 4.45458640e-12],
       [1.83987700e-06, 4.25337900e-07, 9.99997740e-01],
       [5.18638700e-09, 1.00000000e+00, 1.88011250e-09],
       [9.99990460e-01, 9.44454600e-06, 6.42722000e-08],
       [1.00000000e+00, 4.27983480e-08, 2.55972920e-11],
       [7.73610300e-08, 4.28021800e-08, 9.99999900e-01],
       [7.73610300e-08, 4.28021

In [98]:
ovo8 = rocauc_ovo(y_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.9882445759368835

In [99]:
ovr8 = rocauc_ovr(y_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.9882445759368835

In [100]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.9744970414201183

In [101]:
np.std(ovos2)

0.007997510787651003

In [102]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.9744970414201183

In [103]:
np.std(ovrs2)

0.007997510787651003

In [177]:
accs_reg = [acc_test_over5, acc_test_over6, acc_test_over7, acc_test_over8]

In [178]:
mean_reg = np.mean(accs_reg)
print('over-sampling test accuracy regularization mean: %.2f%%' % (mean_reg*100))

over-sampling test accuracy regularization mean: 90.64%


In [179]:
std_reg = np.std(accs_reg)
print('over-sampling test accuracy regularization standard deviation:', std_reg)

over-sampling test accuracy regularization standard deviation: 0.007584717218556131


In [180]:
accs_train_reg = [np.mean(hist1_over5.history['accuracy']), np.mean(hist1_over6.history['accuracy']), np.mean(hist1_over7.history['accuracy']),
             np.mean(hist1_over8.history['accuracy'])]

In [181]:
mean_train_reg = np.mean(accs_train_reg)
print('over-sampling train accuracy regularization mean: %.2f%%' % (mean_train_reg*100))

over-sampling train accuracy regularization mean: 75.47%


In [182]:
std_train_reg = np.std(accs_train_reg)
print('over-sampling train accuracy regularization standard deviation:', std_train_reg)

over-sampling train accuracy regularization standard deviation: 0.004369892
